<img src="https://raw.githubusercontent.com/LovinSpoonful/DATA620-Project2/main/best-star-wars-planets.png?token=GHSAT0AAAAAABRO24QCG3SOAJ3GDP43HEPCYRLQ3ZA" width="100%" style="margin-left:auto; margin-right:auto">

<br>

# Project 2

DATA 620
Bonnie Cooper, George Cruz Deschamps, Rob Hodde.

<br>

## Assignment Information

* Identify a large 2-node network dataset—you can start with a dataset in a repository.  Your data should meet the criteria that it consists of ties between and not within two (or more) distinct groups.
* Reduce the size of the network using a method such as the island method described in chapter 4 of social network analysis.
* What can you infer about each of the distinct groups?
 

<br>

## Environment

In [9]:
import pandas as pd
import pickle
from pathlib import Path
import urllib
from tqdm import tqdm
import os
import networkx as nx
import matplotlib.pyplot as plt
import sqlalchemy

In [18]:
#!pip install ipython-sql
%load_ext sql

<br>

## Identify and load the Wookieepedia network dataset

For this assignment we will be visualizing a Star Wars data set. This dataset was generated by Dennis Bakhuis from web scrapings of the Wookieepedia website. [Wookieepedia](https://starwars.fandom.com/wiki/Main_Page) is the Star Wars wiki page which serves as an online encyclopedia for any and all things about the Star Wars universe. A full description of Dennis's methods are available on [his github](https://github.com/dennisbakhuis/wookieepediascience).  

### Star Wars network Nodes

To begin, we will load a dataframe of Star Wars characters. These charcters will eventually become the nodes of the Star Wars network:

In [10]:
# read in the StarWars_Characters dataframe
characters = pd.read_parquet('data/parquet/StarWars_Characters.parquet')

In [4]:
print( characters.shape )
characters.head()

(5334, 14)


,name,key,url,description,species_2nd,species_3rd,species,home_world,gender,height,eye_color,skin_color,hair_color,weight
0,1138 (First Order),1138_(First_Order),https://starwars.fandom.com/wiki/1138_(First_O...,1138 was a human male stormtrooper who served ...,None,None,Human,None,Male,NaN,None,None,None,None
1,1151,1151,https://starwars.fandom.com/wiki/1151,1151 was a clone cadet who was a part of the C...,None,None,Human,Kamino,Male,NaN,Brown,Tan,Black,None
2,1174,1174,https://starwars.fandom.com/wiki/1174,"1174, nicknamed ""Whiplash,"" was a clone troope...",None,None,Human,Kamino,Male,NaN,Brown,Tan,Black,None
3,224 (Coruscant Security Force),224_(Coruscant_Security_Force),https://starwars.fandom.com/wiki/224_(Coruscan...,224 was the designation of a human male office...,None,None,Human,None,Male,NaN,None,Light,None,None
4,3-6,3-6,https://starwars.fandom.com/wiki/3-6,3-6 was a male death trooper in DT-F16's squad...,None,None,Human,None,Male,NaN,None,None,None,None


<br>  

### Star Wars network edges

Next, we will repeat the data cleansing process from Project 1 to access each character's Crosslinks:

In [11]:
# go through 6 different pickle files downloaded from Dennis' github
files = sorted(Path('data/pickles').glob('*.pickle'))
data = {}
# for each of the 6 files: open, read & add to the data dictionary
for fn in files:
    with open(fn, 'rb') as f:
        part = pickle.load(f)
    data.update(part)

# a helper function to format urls
def remove_url_shizzle(text):
    return urllib.parse.unquote(text).replace('"', '').replace("'", '')

cleaned = {}
#for every key/value pair in data
for key, value in tqdm(data.items()):
    #let's make a niver to read key
    new_key = remove_url_shizzle(key)
    #set value (a dict) as the values for the new cleaned key
    cleaned[new_key] = value
    #format the 'crosslinks' key
    cleaned[new_key]['crosslinks'] = [remove_url_shizzle(crosslink) for crosslink in value['crosslinks']]
data = cleaned

100%|██████████| 29860/29860 [00:02<00:00, 14697.73it/s]


Here are the number of record and list of keys in the data dictionary:

In [12]:
print('Total records in dictionary:', len(data.keys()))
print('Keys:', data['Yoda'].keys() )

Total records in dictionary: 29858
Keys: dict_keys(['url', 'title', 'is_character', 'side_bar', 'paragraph', 'crosslinks'])


Here are the Crosslinks for the character Yoda:

In [13]:
print(data['Yoda']['crosslinks'] )

['The_Gathering', 'Behavioral_modification_biochip', 'Akar-Deshu', 'CT-5555', 'Alliance_to_Restore_the_Republic', 'Chewbacca', 'Darth', 'Ezra_Bridger', 'Jedi', 'Army', 'Starlight_Beacon', 'Immortality', 'Jedi_Knight', 'Nightsisters', 'Skywalker_family', 'Lothal', 'Veter', 'Swamp', 'Count', 'Cloning', 'Huyang', 'Battle_of_Endor', 'Separatist_Droid_Army', 'Kamino', 'Kashyyyk', 'Mandalorian', 'Homeworld', 'Mission_to_Raxus', 'Cave_of_Evil', 'CC-4477', 'Yodas_species', 'Medical_droid', 'Jabba_Desilijic_Tiure', 'Zillo_Beast', 'Second_battle_of_Christophsis', 'Millennium', 'Even_Piell', 'Coleman_Trebor', 'Gas_giant', 'Seeing_stone', 'New_Order', 'Silman', 'Rig_Nema', 'Siege_of_Mandalore', 'Tiplar', 'Battle_of_Kashyyyk', 'Galactic_Empire', 'Obi-Wan_Kenobi', 'Kachirho', 'Force-bond', 'Hutt', 'Bombing_of_the_Jedi_Temple_hangar', 'Plo_Koon', 'Jetpack', 'Firefly', 'Light_side_of_the_Force', '4_ABY', 'Battle_of_Crait', 'Telekinesis', 'Canon', 'Lothal_Jedi_Temple', 'Sith_Lord', 'Letta_Turmond', 'Mi

<br>  

### Star Wars Planets

For Project 2, we want to find **planets** in the Crosslinks, so that we can attach Star Wars characters to Star Wars planets, and create a bipartite graph of Characters And Planets.

But how do we find planetary data? We must study the Star Wars Canon pickle files again.

As an aside, sometimes data engineers must build integrations between Python scripts and on-premise SQL databases, so we will demonstrate one integration method in this section.

The code block below moves the pickle files into the database: 

In [25]:
## copy the pickle files to MS SQL 
sw = sorted(Path('data/pickles').glob('*.pickle'))
params = urllib.parse.quote_plus(r'DRIVER={SQL Server};SERVER=DESKTOP-FD4H0K5;DATABASE=CUNY;Trusted_Connection=yes')
conn_str = 'mssql+pyodbc:///?odbc_connect={}'.format(params)
engine = sqlalchemy.create_engine(conn_str)

print(str(engine))
for i in sw:
   filename = os.path.split(i)[-1]
   d = pd.read_pickle(i) #read pickle into data dictionary
   df = pd.DataFrame.from_dict(d, orient='index').reset_index(drop=False)
   #convert all columns to strings and write to MS SQL
   df.astype(str).to_sql(name=filename,con=engine, if_exists='append',index=False) 
   

Engine(mssql+pyodbc:///?odbc_connect=DRIVER%3D%7BSQL+Server%7D%3BSERVER%3DDESKTOP-FD4H0K5%3BDATABASE%3DCUNY%3BTrusted_Connection%3Dyes)


OperationalError: (pyodbc.OperationalError) ('08001', '[08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]SQL Server does not exist or access denied. (17) (SQLDriverConnect); [08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionOpen (Connect()). (53)')
(Background on this error at: https://sqlalche.me/e/14/e3q8)

Below is a sample of the last pickle file converted to a dataframe:

In [26]:
df.head()

,index,url,title,is_character,side_bar,paragraph,crosslinks
0,The_Grand_Inquisitor%27s_lightsaber,https://starwars.fandom.com/wiki/The_Grand_Inq...,The Grand Inquisitor's lightsaber,False,{'Production information': {'Model': 'Double-b...,The Grand Inquisitor's lightsaber was the pers...,"[Mustafar_system, Jedi_hunter, Order_66, Cumbe..."
1,The_Grievous_Legion,https://starwars.fandom.com/wiki/The_Grievous_...,The Grievous Legion,False,{'General information': {'Formation type': 'Ba...,The Grievous Legion was a battle group in the ...,"[Salissian_special_forces, Corporate_Alliance_..."
2,The_Hammer_of_Ryloth,https://starwars.fandom.com/wiki/The_Hammer_of...,The Hammer of Ryloth,False,{},"""The Hammer of Ryloth"" was a title earned by t...","[Twi%27lek, Battle_of_Ryloth, General, Cham_Sy..."
3,The_Harch,https://starwars.fandom.com/wiki/The_Harch,The Harch,True,"{'Physical description': {'Species': 'Harch', ...","""The Harch"" was the nickname of a female Harch...","[Battle_of_Endor, Sexes, Galactic_Empire, Spac..."
4,The_Haunted_Lands,https://starwars.fandom.com/wiki/The_Haunted_L...,The Haunted Lands,False,{'General information': {'Location': 'Dathomir...,The Haunted Lands were a region of the planet ...,"[Oteroa_Zero-Twenty, Dathomir, Planet, Canon]"


Here is a screen capture of the new SQL Database tables taken from SQL Server Management Studio:


<img src="https://raw.githubusercontent.com/LovinSpoonful/DATA620-Project2/main/SQL-Pickle-Tables.png" width="33%" style="margin-left:auto; margin-right:auto">


Below is a screenshot of a few records from one of the converted pickle tables:



<img src="https://raw.githubusercontent.com/LovinSpoonful/DATA620-Project2/main/SQL-Pickle-Rows.png" width="100%" style="margin-left:auto; margin-right:auto">


We combine the six tables into one:

<img src="https://raw.githubusercontent.com/LovinSpoonful/DATA620-Project2/main/SQL-Pickle-Combine.png" width="33%" style="margin-left:auto; margin-right:auto">


Since we are looking for planets, we can query on the name of a known planet. Note the "Sidebar" column contains "Astrological information" and the Paragraph contains the word "planet".  We will use this as a starting point to search for planets. 

<img src="https://raw.githubusercontent.com/LovinSpoonful/DATA620-Project2/main/Canon-Jakku.png" width="60%" style="margin-left:auto; margin-right:auto">

We add column "ObjectType" to store our classifications. This helps if some must be classified manually due to dirty data.



<img src="https://raw.githubusercontent.com/LovinSpoonful/DATA620-Project2/main/Canon-ObjectType.png" width="70%" style="margin-left:auto; margin-right:auto">

The ObjectType column also helps us declassify "false matches":

<img src="https://raw.githubusercontent.com/LovinSpoonful/DATA620-Project2/main/Astro-NotPlanets.png" width="70%" style="margin-left:auto; margin-right:auto">

**OK, that's enough SQL!   How to get the data back to Python?**  

We save it as a parquet!

In [11]:
import pyodbc
import fastparquet

# define the database connection
sServer = 'localhost'
sDB = 'CUNY'
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                     "Server=" + sServer + ";"
                     "Database=" + sDB + ";"
                     "Trusted_Connection=yes;")  

#export list of 826 planets from SQL to parquet
sSQL = "SELECT ObjectName Name FROM tbl_StarWars_Canon WHERE ObjectType = 'Planet'" # query
df = pd.read_sql_query(sSQL, cnxn) #read the SQL data into a pandas dataframe
df.to_parquet('StarWars_Canon_All_Planets.parquet', engine='fastparquet') #convert the dataframe to a parquet file

# We also made a medium-sized list of planets, sourced from https://en.wikipedia.org/wiki/List_of_Star_Wars_planets_and_moons
sSQL = "SELECT Name FROM tbl_StarWars_Planets" 
df = pd.read_sql_query(sSQL, cnxn)
df.to_parquet('StarWars_Planets.parquet', engine='fastparquet')

# We also made a small list of planets, sourced from https://screenrant.com/star-wars-rogue-one-planets-ranked-jedha-scarif-tatooine/
sSQL = "SELECT Name FROM tbl_StarWars_Planets WHERE Description2 IS NOT NULL" 
df = pd.read_sql_query(sSQL, cnxn)
df.to_parquet('StarWars_Planets_Shortlist.parquet', engine='fastparquet')


We see the parquet files now in our Github repo directory:


<img src="https://raw.githubusercontent.com/LovinSpoonful/DATA620-Project2/main/Planets-Parquets.png" width="50%" style="margin-left:auto; margin-right:auto">




<br>  
Next we want to join our Star Wars characters to Star Wars Planets.  We read the parquets into lists:

In [12]:
characters_list = pd.read_parquet('StarWars_Characters.parquet')['key'].tolist()

df = pd.read_parquet('StarWars_Canon_All_Planets.parquet') 
planets_large_list = df['Name'].to_list()

df = pd.read_parquet('StarWars_Planets.parquet') 
planets_medium_list = df['Name'].to_list()

df = pd.read_parquet('StarWars_Planets_Shortlist.parquet') 
planets_short_list = df['Name'].to_list()


<br>  

### Graph Objects: Characters and Planets

Next we will create a network graph object, linking all Star Wars characters to all planets: 

In [15]:
#create a graph object for all characters, all planets
graph_large = nx.Graph()
#loop thru each character
for key in tqdm(characters_list):  #tqdm = progress bar
    #use this key from characters to get the crosslinks from data
    crosslinks = data[key]['crosslinks']
    for crosslink in crosslinks:
        # if the crosslink is to a planet, 
        # then this connection describes a connection between a character and a planet
        if crosslink in planets_large_list:
            # add to the .Graph object
            graph_large.add_edge(key, crosslink)
        
print(f'Nodes: {graph_large.number_of_nodes()}, Links: {graph_large.number_of_edges()}')      

100%|██████████| 5334/5334 [00:01<00:00, 4151.59it/s]

Nodes: 4224, Links: 7927


Here are the first ten nodes (characters):

In [17]:
# list a few nodes
print(list(graph_large.nodes)[0:10] )


['1138_(First_Order)', 'Jakku', '1151', 'Kamino', '1174', '224_(Coruscant_Security_Force)', 'Coruscant', '3-9', 'Lothal', '7173']
[('Saw_Gerrera', 'Kashyyyk'), ('Saw_Gerrera', 'Jedha'), ('Saw_Gerrera', 'Jeosyn'), ('Saw_Gerrera', 'Onderon'), ('Saw_Gerrera', 'Yerbana'), ('Saw_Gerrera', 'Alderaan'), ('Saw_Gerrera', 'Agamar'), ('Saw_Gerrera', 'Geonosis'), ('Saw_Gerrera', 'Serenno'), ('Saw_Gerrera', 'Inusagi'), ('Saw_Gerrera', 'Core_Worlds'), ('Saw_Gerrera', 'Wrea'), ('Saw_Gerrera', 'Coruscant')]


Here are the planets associated with the character Saw Gerrera:

In [18]:
# list edges for one character - these are the planets associated with that character
print(graph_large.edges('Saw_Gerrera') )

[('Saw_Gerrera', 'Kashyyyk'), ('Saw_Gerrera', 'Jedha'), ('Saw_Gerrera', 'Jeosyn'), ('Saw_Gerrera', 'Onderon'), ('Saw_Gerrera', 'Yerbana'), ('Saw_Gerrera', 'Alderaan'), ('Saw_Gerrera', 'Agamar'), ('Saw_Gerrera', 'Geonosis'), ('Saw_Gerrera', 'Serenno'), ('Saw_Gerrera', 'Inusagi'), ('Saw_Gerrera', 'Core_Worlds'), ('Saw_Gerrera', 'Wrea'), ('Saw_Gerrera', 'Coruscant')]


Next we need to integrate the "Sith" and "Jedi" (Bad/Good) polarity into the dataset, so that we can create a bipartite graph

Sith = bipartite=0
Jedi = bipartite=1



In [19]:
from networkx.algorithms import bipartite


